In [10]:
from google.colab import drive
drive.mount("/content/drive")
import nltk
nltk.download('punkt')
import numpy as np
!pip install -U sentence-transformers
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate
import xgboost as xgb
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from nltk.util import ngrams
import re
from pathlib import Path
import string
from sklearn.model_selection import train_test_split
from functools import reduce
from math import log
import itertools
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score

import json
from nltk.tokenize.casual import TweetTokenizer
import pickle
from gensim.models.word2vec import Word2Vec
import keras
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.metrics import precision_recall_fscore_support
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
import gensim.downloader as api
import gensim
from sklearn.model_selection import KFold

t = TweetTokenizer()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
dir="/content/drive/MyDrive/nlp_project/"
os.chdir(dir)
df=pd.read_csv("train.csv")
output=list(df["is_duplicate"])
output_np=np.array(output)

In [3]:

output_encode=to_categorical(output_np)
output_encode=np.array(output_encode)


In [4]:
print(output_encode[:40])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [5]:
open_file = open("embeddings_para_1", "rb")
embeddings1= pickle.load(open_file)
open_file.close()
open_file = open("embeddings_para_2", "rb")
embeddings2 = pickle.load(open_file)
open_file.close()

In [6]:
embeddings1=np.array(embeddings1)
embeddings2=np.array(embeddings2)

In [7]:
input=np.hstack((embeddings1,embeddings2))

In [8]:
print(input.shape)

(404348, 768)


In [9]:
accuracy=[]
loss=[]
predictions=[]
truth=[]
X_train, X_test, y_train, y_test = train_test_split(input, output_encode, test_size=0.25, random_state=42)
model = Sequential()
#model.add(Dense(1024, input_dim=768,activation='relu'))
model.add(Dense(512, input_dim=768,activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['acc'])
model.fit(X_train,y_train,epochs=6,validation_split=0.2,batch_size=64)
score=model.evaluate(X_test,y_test)
print(score[1])
accuracy.append(score[1])
loss.append(score[0])

y_pred = model.predict(X_test)
y_pred= np.argmax(y_pred,axis=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               393728    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 16)                1040      
                                                                 
 dense_5 (Dense)             (None, 2)                 34        
                                                                 
Total params: 567,282
Trainable params: 567,282
Non-trai

In [13]:
y_test=np.argmax(y_test,axis=1)

In [14]:
print("S_Bert+MLP")
print("Accuracy: ",accuracy_score(y_test, y_pred)*100)
print("Classification report:\n", classification_report(y_test, y_pred))
print("F1 Score:\n ",f1_score(y_test, y_pred)*100)

S_Bert+MLP
Accuracy:  82.43592153293697
Classification report:
               precision    recall  f1-score   support

           0       0.86      0.86      0.86     64078
           1       0.76      0.76      0.76     37009

    accuracy                           0.82    101087
   macro avg       0.81      0.81      0.81    101087
weighted avg       0.82      0.82      0.82    101087

F1 Score:
  75.91726008816548
